In [ ]:
stack_prefix = 's1-'

In [ ]:
cf_bucket = 'ds-deploy-us-east-1'
template_name = 'smssub-deploy.yml'
smssub_stack = stack_prefix + 'smssub'
smssub_zip = 'smssub-deployment.zip'

In [ ]:
%%bash -s "$template_name" "$smssub_zip" "$cf_bucket"
aws s3 cp ./$1 s3://$3
aws s3 cp ./$2 s3://$3

In [ ]:
import boto3

client = boto3.client('cloudformation')

In [ ]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [ ]:
import os
region = os.environ['AWS_DEFAULT_REGION']
bucketRoot = form_s3_url_prefix(region) + '/' + cf_bucket
print bucketRoot

In [ ]:
response = client.create_stack(
    StackName=smssub_stack,
    TemplateURL=bucketRoot + '/' + template_name,
    Parameters=[
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': cf_bucket
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
# Wait for it...
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=smssub_stack
)

print 'stack created'

## Clean Up

In [ ]:
print 'destroy {}'.format(smssub_stack)
response = client.delete_stack(
    StackName=smssub_stack
)

print response

print 'waiting for destroy of {}...'.format(smssub_stack)
waiter.wait(
    StackName=smssub_stack
)

print 'Stack {} destroyed'.format(smssub_stack)